In [2]:
from geo_script import *
from Matching.matching import get_p3b_list, write_grid3_csv, write_rrcollect_csv, get_captured_list
from helper import kwara_lga_map, kwara_wards_map, wards_shapefile
# from Tracker.tracker import create_to_capture_dict, match
import pandas as pd

# file_name = "merged_list.xlsx"
fl2 = "RR_Collect_Merged_Form_Data (21).xlsx"
# fl3 = "Nigeria_-_Settlement_Points.csv"
df1 = pd.read_excel(fl2)
settlement_file_name, dh_file_name = write_rrcollect_csv(df1,True)
print(settlement_file_name)
# df = pd.read_csv(settlement_file_name)
# print(df.head(20))
uri= f'file:///{settlement_file_name}?delimiter=,&yField=Latitude&xField=Longitude'
settlement_layer = QgsVectorLayer(uri, 'settlement', 'delimitedtext')
memory_layer = QgsVectorLayer("Point?crs=EPSG:4326", 'memory_layer', 'memory')

# Add the same fields to the memory layer
memory_layer.dataProvider().addAttributes(settlement_layer.fields())
memory_layer.updateFields()

# Copy features from the delimited text layer to the memory layer
memory_layer.startEditing()
memory_layer.dataProvider().addFeatures(settlement_layer.getFeatures())
memory_layer.commitChanges()
# for layer in settlement_layer.getFeatures():
#     print(layer)
wards_layer = QgsVectorLayer(wards_shapefile, "wards", "ogr")
query = f'"statename"  =  \'Kwara\''
wards_layer = crt_subset_lyr(wards_layer,query)
# for layer in wards_layer.getFeatures():
#     print(layer.attributes())
new_settlement_layer = within_ward_boundary(memory_layer,wards_layer,kwara_lga_map,kwara_wards_map)
for layer in new_settlement_layer.getFeatures():
    print(layer.attributes())
# df2 = pd.read_excel(file_name)
# lgas = list(df2.LGA.unique())
# # for lga in lgas[0:1]:
#     # print(lga)
#     # dict1 = get_captured_list(cleand_df[0],lga,{})

# dict1, dict2 = create_to_capture_dict(lgas,df2,cleand_df[0])
# # print(dict1)
# # print(dict2)
# match(dict1,dict2)



c:\Users\ASUS\anaconda3\envs\GIS\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


c:\Users\ASUS\ITN\p3b\Cleaned_adamawa_settlement_capture.csv
['State', 'LGA', 'Ward', 'Name of Settlement', 'DH', 'Latitude', 'Longitude', 'Acurracy', 'Altitude', 'Date_time']
['State', 'LGA', 'Ward', 'Name of Settlement', 'DH', 'Latitude', 'Longitude', 'Acurracy', 'Altitude', 'Date_time', 'in_Ward', 'dst_km', 'GRID3_Wrd']
<QgsCoordinateReferenceSystem: EPSG:4326>
meters
<QgsGeometry: Point (4.47542920999999971 8.40207082999999955)>
hey
<QgsGeometry: Point (4.51694998999999964 8.49981307000000008)>
<QgsGeometry: Point (4.51699377000000002 8.4997694500000005)>
<QgsGeometry: Point (4.59184999999999999 8.41733666670000069)>
<QgsGeometry: Point (4.38831776999999956 8.31126395999999978)>
<QgsGeometry: Point (4.53336633330000005 8.47989339999999991)>
<QgsGeometry: Point (5.42008500000000026 8.85435666669999932)>
hey
<QgsGeometry: Point (5.41986666669999995 8.85495333329999923)>
hey
<QgsGeometry: Point (5.42024500000000042 8.85543666669999929)>
hey
<QgsGeometry: Point (4.53394500000000011 8.4

In [ ]:
from script import *
file_name = "2021_Jigawa_P3B.xlsx"
sheets = get_sheets(file_name)
# print(re.match(r'^sub\s*total\s*$',"subtotal",re.IGNORECASE))
for sheet in sheets[1:2]:
    df = pd.read_excel(file_name,sheet)
    header = actual_header_row(df)
    df = remove_first_blank_column(file_name,header,sheet)
    group = []
    ward =""
    # print(df.head(60))
    pop = populate_wards(df)
# print(pop[["Wards","List of contiguous communities/ settlements"]].head(60))

In [ ]:
# file_name = "2021_Jigawa_P3B.xlsx"
# sheets = get_sheets(file_name)
# print(re.match(r'^sub\s*total\s*$',"subtotal",re.IGNORECASE))
# print(pop.columns)
# print(pop[['List of contiguous communities/ settlements',"Name of DH"]].head(60))
df =populate_dh(pop)
# print(df[['List of contiguous communities/ settlements',"Name of DH"]].head(60))


print(df[["Name of DH",'List of contiguous communities/ settlements']].head(60))


0                                 Name of DH  \
0                                              
1                                              
2                                              
3                                        NaN   
4          Tsohon Asibiti Village Head House   
5          Tsohon Asibiti Village Head House   
6          Tsohon Asibiti Village Head House   
7                                        NaN   
8             Makara Huta Village Head House   
9             Makara Huta Village Head House   
10            Makara Huta Village Head House   
11                                       NaN   
12         Babura K/Gabas Village Head House   
13         Babura K/Gabas Village Head House   
14         Babura K/Gabas Village Head House   
15                                       NaN   
16         Akula Quarters Village Head House   
17         Akula Quarters Village Head House   
18                                       NaN   
19             Sanin Anjo Village Head H

In [ ]:
sheets = get_sheets(file_name)
lga_dct={}
text="{"
for sheet in sheets:
    lga = sheet.split('.')[1].strip().lower()
    print(lga)
    demo_df = pd.read_excel(file_name, sheet_name=sheet)
    row_index = actual_header_row(demo_df)
    clean_data = remove_first_blank_column(file_name, row_index, sheet)
    new_data = remove_unwanted_columns(clean_data, needed_columns)
    base_data = remove_blank_wards_rows(new_data)
    ward_list = []
    # if lga in kwara_security_challenged:
    #     ward_list = kwara_security_challenged[lga]
    wards_df = drop_subtotal_rows(base_data, ward_list)
    ward_dct={}
    for idx in range(len(wards_df)):
        ward = str(wards_df["Wards"][idx]).lower().strip()
        ward_dct[ward]=kwara_wards_map[ward]
    lga_dct[lga]=ward_dct
    # print(lga_dct)
    text+=f"'{lga}': {lga_dct[lga]}\n"
    lga_dct= {}
text+="}"
print(text)

In [ ]:
r.text
"https://www.google.com/maps/dir/'8.456104,4.544522'/9.074566390856697,4.968173660214374"
https://www.google.com/maps/dir/?api=1&origin=4.544522,8.456104&destination=4.968173660214374,9.074566390856697

'{"type":"FeatureCollection","metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"routing","timestamp":1698385665043,"query":{"coordinates":[[4.544522,8.456104],[4.956416828517544,9.065823591445978]],"profile":"driving-car","format":"json"},"engine":{"version":"7.1.0","build_date":"2023-07-09T01:31:50Z","graph_date":"2023-10-22T10:16:37Z"}},"features":[{"bbox":[4.544373,8.455836,4.956431,9.066795],"type":"Feature","properties":{"transfers":0,"fare":0,"segments":[{"distance":95097.8,"duration":5528.9,"steps":[{"distance":24.1,"duration":1.3,"type":11,"instruction":"Head southwest","name":"-","way_points":[0,1]},{"distance":12.1,"duration":4.3,"type":0,"instruction":"Turn left","name":"-","way_points":[1,2]},{"distance":3330.8,"duration":196.1,"type":0,"instruction":"Turn left","name":"-","way_points":[2,41]},{"distance":209.6,"duration":17.9,"type":0,"instruction":"Turn left onto Unity Road","name":"Unity Road","way_points":[41,48]},{"distance":1050.7

In [ ]:
processing.algorithmHelp("native:clip")
# parameters = { 'INPUT' : state_extent, 
#                'VALUE' : 1, 
#                 }
# point =processing.run('native:selectbylocation', parameters)
# /algs = dict()
# for alg in QgsApplication.processingRegistry().algorithms():
#     algs[alg.displayName()] = alg.id()
#     print(alg.displayName(), alg.id() )

Clip (native:clip)

This algorithm clips a vector layer using the features of an additional polygon layer. Only the parts of the features in the Input layer that fall within the polygons of the Overlay layer will be added to the resulting layer.

The attributes of the features are not modified, although properties such as area or length of the features will be modified by the clipping operation. If such properties are stored as attributes, those attributes will have to be manually updated.


----------------
Input parameters
----------------

INPUT: Input layer

	Parameter type:	QgsProcessingParameterFeatureSource

	Accepted data types:
		- str: layer ID
		- str: layer name
		- str: layer source
		- QgsProcessingFeatureSourceDefinition
		- QgsProperty
		- QgsVectorLayer

OVERLAY: Overlay layer

	Parameter type:	QgsProcessingParameterFeatureSource

	Accepted data types:
		- str: layer ID
		- str: layer name
		- str: layer source
		- QgsProcessingFeatureSourceDefinition
		- QgsProperty
	